# Rossman Stors Sales Analysis Using Linear Regression

## Problem Statement / Business Understanding

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to **six weeks** in advance. 

Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

In this exercise, we are going to **predict 6 weeks of daily sales for 1,115 stores located across Germany**. 

Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation. By helping Rossmann create a robust **prediction** model, we will help store managers stay focused on what’s most important to them: their customers and their teams!

### Hypothesis Statement

H0: There is no direct correlation between numbers of Rossmann customers and volumes of sales at the stores.

H1: The number of customers influence the volumes of sales at Rossmann stores. The greater the number of customers, the higher the volume of sales. The less the number of customers, the lower the volume of sales.

## Data Understanding

We have historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

### Files

- `train.csv` - historical data including Sales
- `test.csv` - historical data excluding Sales
- `sample_submission.csv` - a sample submission file in the correct format
- `store.csv` - supplemental information about the stores

## Data Preparation

### Loading Modules

In [1]:
# Imports

import matplotlib as plt
import missingno as msno
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import sklearn as sk
import warnings


# suppress scientific notation in Pandas
pd.options.display.float_format = "{:,.2f}".format
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", None)

# Matplotlib settings
plt.rcParams["figure.figsize"] = [12, 10]
plt.rcParams["figure.dpi"] = 150

# Seaborn settings
sns.set_context("notebook", rc={"lines.linewidth": 2.5})
sns.set_style("whitegrid")
sns.set_theme(style="darkgrid")

# Ignore warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 1

### Loading Training Data

In [2]:
df = pd.read_csv("./sales_data/train.csv")

df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


### Inspecting Training Data

In [3]:
df.shape

(1017209, 9)

The training data has a total of 1,017,209 records and 9 columns. We need to inspect what this data looks like by checking the first 5 and the last 5 records. This data is not sorted yet.

In [4]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
df.tail()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1
1017208,1115,2,2013-01-01,0,0,0,0,a,1


We need to see more information about the dataframe, including the index dtype and columns, non-null values and memory usage. [Ref](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html#pandas.DataFrame.info)

In [7]:
df.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 158.3 MB


Some descriptive statistics, excluding string columns [Ref](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html#pandas.DataFrame.describe)

In [10]:
df.describe(include=[np.number])

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,"1,017,209.00","1,017,209.00","1,017,209.00","1,017,209.00","1,017,209.00","1,017,209.00","1,017,209.00"
mean,558.43,4.00,"5,773.82",633.15,0.83,0.38,0.18
std,321.91,2.00,"3,849.93",464.41,0.38,0.49,0.38
min,1.00,1.00,0.00,0.00,0.00,0.00,0.00
25%,280.00,2.00,"3,727.00",405.00,1.00,0.00,0.00
50%,558.00,4.00,"5,744.00",609.00,1.00,0.00,0.00
75%,838.00,6.00,"7,856.00",837.00,1.00,1.00,0.00
max,"1,115.00",7.00,"41,551.00","7,388.00",1.00,1.00,1.00


## Data Modelling

## Conclusions

## Next Steps